In [3]:
# Import modules
import Hyper_parameter_search
from Hyper_parameter_search import Smoothing_method,weights_init_normal,HyperParameters,Hyperparameter_Search,Hyperparameter_Test,Hyper_parameter_GridSearch,FCNN,Compile_class,Compile_train

import inspect
import random
import torch
import torch.nn.init as init
import torch.nn.functional as F
import torch.nn as nn

from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import numpy as np
from numpy import *
import seaborn as sns
import matplotlib
# matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
import skfda as fda
from skfda import representation as representation
from skfda.exploratory.visualization import FPCAPlot
from skfda.exploratory.visualization import FPCAPlot
from skfda.preprocessing.dim_reduction import FPCA
from skfda.representation.basis import BSpline, Fourier, Monomial
import scipy
from scipy.interpolate import BSpline
import os
import ignite
from tqdm import tqdm
import sklearn
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
import random
from random import seed
from scipy import stats
import statistics
from statistics import stdev
import gc
import skfda
from skfda import FDataGrid as fd
from skfda.representation.basis import BSpline as B


In [15]:
Phoneme=skfda.datasets.fetch_cran("phoneme",package_name="fda.usc")
phoneme=Phoneme['phoneme']

x_train,x_test,y_train,y_test=phoneme['learn'],phoneme['test'],phoneme['classlearn'],phoneme['classtest']
x_train.shape,x_test.shape,y_train.categories,y_test.categories
##Phonèmes enregistrés à classer dans 5 catégories

c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:321: UserWarning: Unknown encoding. Assumed ASCII.
  warnings.warn("Unknown encoding. Assumed ASCII.")
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:321: UserWarning: Unknown encoding. Assumed ASCII.
  warnings.warn("Unknown encoding. Assumed ASCII.")


((250,),
 (250,),
 Index(['1', '2', '3', '4', '5'], dtype='object'),
 Index(['1', '2', '3', '4', '5'], dtype='object'))

In [6]:

x_train_tensor=torch.tensor(x_train.data_matrix).reshape(250,1,150).cuda().float()
x_test_tensor=torch.tensor(x_test.data_matrix).reshape(250,1,150).cuda().float()
y_train_tensor=torch.tensor(y_train.codes).unsqueeze(1).unsqueeze(2).long().cuda()
y_test_tensor=torch.tensor(y_test.codes).unsqueeze(1).unsqueeze(2).long().cuda()

X=torch.cat([x_train_tensor,x_test_tensor])
Y=torch.cat([y_train_tensor,y_test_tensor])


Choix de la méthode de smoothing on créer une instance de la classe Smoothing_method(), ici par exemple on choisit la base de Bspline et un smoothing au lieu de l'interpolation. On choisit la base avec les noeufs et l'ordre de spline, à noter que l'on peut également le choisir avec le nombre de fonctions de basis en mettant les knots_or_basis="basis" et n_basis=..., par défaut knots_or_basis="knots")
 

In [20]:
smoother=Smoothing_method(Mode='Smoothing',basis_type="Bspline",knots=np.linspace(1,12,8),B_spline_order=4)


Création des hyperparamètres: Le NN a trois couches et les paramètres de chaque couche sont modifiables dans l'instance de la classe HyperParameters(). 

In [ ]:

params=HyperParameters(negative_slope=0.2,Smoothing_method=smoother 
)
params.basis

In [ ]:
smoother=Smoothing_method(Mode='Smoothing',basis_type="Bspline",knots=np.linspace(1,12,8),B_spline_order=4)

params=HyperParameters(negative_slope=0.2,Smoothing_method=smoother 
)
params.basis

In [17]:

class TSCNN(nn.Module):
    def __init__(self, hyperparams,output_size):
        super(TSCNN, self).__init__()
        n_conv_out =hyperparams.n_conv_out
        basis = hyperparams.basis
        granulation = hyperparams.granulation
        n_conv_in = hyperparams.n_conv_in
        n_conv_in2 = hyperparams.n_conv_in2
        n_conv_in3 = hyperparams.n_conv_in3
        n_Flat_out = hyperparams.n_Flat_out
        stride_1 = hyperparams.stride_1
        stride_2 = hyperparams.stride_2
        stride_3 = hyperparams.stride_3
        stride_pool_1 = hyperparams.stride_pool_1
        stride_pool_2 = hyperparams.stride_pool_2
        stride_pool_3 = hyperparams.stride_pool_3
        kernel_size_1 = hyperparams.kernel_size_1
        kernel_size_2 = hyperparams.kernel_size_2
        kernel_size_3 = hyperparams.kernel_size_3
        kernel_size_pool_1 = hyperparams.kernel_size_pool_1
        kernel_size_pool_2 = hyperparams.kernel_size_pool_2
        kernel_size_pool_3 = hyperparams.kernel_size_pool_3
        dilation_1 = hyperparams.dilation_1
        dilation_2 = hyperparams.dilation_2
        dilation_3 = hyperparams.dilation_3
        dilation_pool_1 = hyperparams.dilation_pool_1
        dilation_pool_2 = hyperparams.dilation_pool_2
        dilation_pool_3 = hyperparams.dilation_pool_3
        basis=hyperparams.basis
        negative_slope=hyperparams.negative_slope
        padding_1 = hyperparams.padding_1
        padding_2 = hyperparams.padding_2
        padding_3 = hyperparams.padding_3
        padding_pool_1 = hyperparams.padding_pool_1
        padding_pool_2 = hyperparams.padding_pool_2
        padding_pool_3 = hyperparams.padding_pool_3
        negative_slope = hyperparams.negative_slope
        
        # Reste du code pour l'initialisation de la classe model
        self.basis=basis
        self.granulation=granulation
        self.convlayer1=nn.Sequential(
            nn.Conv1d(1,n_conv_in,kernel_size=kernel_size_1,stride=stride_1,padding=padding_1,dilation=dilation_1),
            nn.BatchNorm1d(n_conv_in),
            nn.LeakyReLU(negative_slope),
            hyperparams.activation,
            nn.MaxPool1d(kernel_size=kernel_size_pool_1,stride=stride_pool_1,padding=padding_pool_1,dilation=dilation_pool_1),
            nn.BatchNorm1d(n_conv_in),
            nn.LeakyReLU(negative_slope),
        )
        
        self.convlayer2=nn.Sequential(
            nn.Conv1d(n_conv_in,n_conv_in2,kernel_size=kernel_size_2,stride=stride_2,padding=padding_2,dilation=dilation_2),
            nn.BatchNorm1d(n_conv_in2),
            nn.LeakyReLU(negative_slope),
            hyperparams.activation,
            nn.MaxPool1d(kernel_size=kernel_size_pool_2,stride=stride_pool_2,padding=padding_pool_2,dilation=dilation_pool_2),
            nn.BatchNorm1d(n_conv_in2),
            nn.LeakyReLU(negative_slope),
        )
        
        self.convlayer3=nn.Sequential(

            nn.Conv1d(n_conv_in2,n_conv_in3,kernel_size=kernel_size_3,stride=stride_3,padding=padding_3,dilation=dilation_3),
            nn.BatchNorm1d(n_conv_in3),
            nn.LeakyReLU(negative_slope),
            hyperparams.activation,
            nn.MaxPool1d(kernel_size=kernel_size_pool_3,stride=stride_pool_3,padding=padding_pool_3,dilation=dilation_pool_3),
            nn.BatchNorm1d(n_conv_in3),
            nn.LeakyReLU(negative_slope),
        )

        self.fc_block=nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_conv_out*n_conv_in3,n_Flat_out),
            nn.BatchNorm1d(n_Flat_out),
            nn.LeakyReLU(negative_slope),
            hyperparams.activation,
            nn.Linear(n_Flat_out,output_size),
            
        )
        self.n_conv_out=n_conv_out
        self.Smoothing_type=hyperparams.Smoothing_type
        self.hyperparameters=hyperparams
        self.smoother=hyperparams.Smoothing_method
        
    def Granulator(self,x):
        ##On choisit ici le mode de smoothing, le premier cas c'est le smoothing
        if "inter" not in self.Smoothing_type:     
        ## Puis on sépare les deux cas selon si l'objet donné est Fdatagrid ou torch.tensor
            if isinstance(x,skfda.representation.grid.FDataGrid):
                eval_points=linspace(1,x.grid_points[0].shape[0],self.granulation)
                coefs=x.to_basis(basis=self.basis).coefficients
                basis_eval=self.basis.evaluate(eval_points=eval_points)[:, :, 0]
                basis_fc = torch.from_numpy(basis_eval).float().cuda()
                
            elif isinstance(x,torch.Tensor):
                eval_points=linspace(1,x.shape[2],self.granulation)
                basis_eval=self.basis.evaluate(eval_points=eval_points)[:, :, 0]
                basis_fc = torch.from_numpy(basis_eval).float().cuda()
                coefs=fd(x[:,0,:].cpu(),grid_points=np.arange(x.shape[2]+1)[1:]).to_basis(basis=self.basis).coefficients

            else:
                raise ValueError("the NN argument must be either torch.tensor or skfda.representation.grid.FDataGrid")
        
                
            coefs_torch=torch.tensor(coefs).float().cuda()
            Recons_train=torch.matmul(coefs_torch,basis_fc)
            Recons_train=Recons_train.reshape(Recons_train.shape[0],1,Recons_train.shape[1])
        ##On choisit ici le mode de smoothing, le deuxième cas c'est l'interpolation 
        ## Ici l'hypothèse faite est que les observations ne comportent pas d'erreur.

        else:## Puis on sépare les deux cas selon si l'objet donné est Fdatagrid ou torch.tensor
            if isinstance(x,skfda.representation.grid.FDataGrid):
                x.interpolation=self.smoother.smoothing()
                eval_points=linspace(1,x.grid_points[0].shape[0],self.granulation)
                Recons_train=x.interpolation._evaluate(fdata=x,eval_points=eval_points)[:,:,0]
                Recons_train=torch.tensor(Recons_train).reshape(Recons_train.shape[0],1,Recons_train.shape[1])
            if isinstance(x,torch.Tensor):
                grid=fd(x[:,0,:].cpu(),grid_points=np.arange(x.shape[2]+1)[1:])
                eval_points=linspace(1,x.shape[2],self.granulation)
                grid.interpolation=self.smoother.smoothing()
                Recons_train=grid.interpolation._evaluate(fdata=grid,eval_points=eval_points)
                Recons_train=torch.tensor(Recons_train).reshape(Recons_train.shape[0],1,Recons_train.shape[1])
            else:
                raise ValueError("the NN argument must be either torch.tensor or skfda.representation.grid.FDataGrid")
        

        return Recons_train.float().cuda()


    def forward(self,x):
        Granulated_x_train=self.Granulator(x)
        tresh_out=torch.relu(Granulated_x_train)
        Conv_out=self.convlayer1(tresh_out)
        Conv_out2=self.convlayer2(Conv_out)
        Conv_out3=self.convlayer3(Conv_out2)
        Lin_out=self.fc_block(Conv_out3)
        return Lin_out.float().unsqueeze_(2).unsqueeze_(3)


Création d'une instance de la classe: creer la classe demande deux choses: Les hyperparamètres (objet de la class HyperParameters) et la taille de la couche finale (ici pour les données phonèmes output_size=5 car il y a 5 catégories)

In [22]:
TSC_model=TSCNN(hyperparams=params,output_size=5).cuda().apply(weights_init_normal)
Granul=TSC_model.Granulator(x_train)
Conv_out=TSC_model.convlayer1(Granul)
Conv_out2=TSC_model.convlayer2(Conv_out)
Conv_out3=TSC_model.convlayer3(Conv_out2)
n_conv_out1=Conv_out.shape[2]
n_conv_out2=Conv_out2.shape[2]
n_conv_out3=Conv_out3.shape[2]
n_conv_out=n_conv_out3
print(n_conv_out3)
params.n_conv_out=n_conv_out
TSC_model=TSCNN(hyperparams=params,output_size=5).cuda().apply(weights_init_normal)
# TSC_model.fc_block,TSC_model.n_conv_out*params.n_conv_in3


C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_21068\1589760509.py:92: DeprecationWarning: The method 'evaluate' is deprecated. Please use the normal calling notation on the basis object instead.
  basis_eval=self.basis.evaluate(eval_points=eval_points)[:, :, 0]


500


In [23]:
opt=params.opt
lr=params.lr
loss=params.loss
batch_size=params.batch_size
betas = [0.5, 0.999]
if opt == "Adam":
    optimizer = optim.Adam(TSC_model.parameters(), lr=lr, betas=betas)
else:
    optimizer = optim.SGD(TSC_model.parameters(), lr=lr)
def train(n_epochs=params.n_epochs, module=TSC_model, optimizer=optimizer, loss=loss, batch_size=batch_size):
    for epoch in tqdm(range(n_epochs)):
        train_loss = torch.tensor(0).cuda().long()
        
        # Mélanger les données d'entraînement
        indices = list(range(len(x_train)))
        random.shuffle(indices)
        
        batch_index = 0  # Indice de batch
        
        for i in range(int(len(x_train) / batch_size)):
            # Obtenir les indices des données mélangées
            batch_indices = indices[batch_index:batch_index+batch_size]
            
            functions_train = x_train[batch_indices]
            labels_train = y_train_tensor[batch_indices]
            
            optimizer.zero_grad()
            output = module(functions_train)
            loss_value = loss(input=output, target=labels_train)
            
            loss_value.backward()
            optimizer.step()
            train_loss += loss_value.long()
            
            batch_index += batch_size  # Passer au prochain batch
            
        return train_loss, loss_value

In [44]:
train(n_epochs=1,module=TSC_model)
accuracy=((torch.sum(torch.argmax(TSC_model(x_test_tensor),dim=1)==y_test_tensor)/x_test_tensor.shape[0])*100)
print("Précision =",accuracy.detach().cpu().numpy(),"%")  


  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_21068\1589760509.py:92: DeprecationWarning: The method 'evaluate' is deprecated. Please use the normal calling notation on the basis object instead.
  basis_eval=self.basis.evaluate(eval_points=eval_points)[:, :, 0]
  0%|          | 0/1 [00:03<?, ?it/s]
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_21068\1589760509.py:97: DeprecationWarning: The method 'evaluate' is deprecated. Please use the normal calling notation on the basis object instead.
  basis_eval=self.basis.evaluate(eval_points=eval_points)[:, :, 0]


Précision = 93.600006 %


In [45]:
accuracy=((torch.sum(torch.argmax(TSC_model(x_test_tensor),dim=1)==y_test_tensor)/x_test_tensor.shape[0])*100)
print("Précision =",accuracy.detach().cpu().numpy(),"%")  

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_21068\1589760509.py:97: DeprecationWarning: The method 'evaluate' is deprecated. Please use the normal calling notation on the basis object instead.
  basis_eval=self.basis.evaluate(eval_points=eval_points)[:, :, 0]


Précision = 93.600006 %
